# Recursive Least Squares Estimation

## Reference
[1] L. Ljung, System Identification Theory for the User

## 1 Linear Regressions and Least Squares
### 1.1 Autoregressive-moving-average Model
考虑自回归滑动平均模型（ARMA），

### 1.2 Least-squares Criterion

为了得到$\theta$，我们首先需要一个判断准则去评定$\theta$的好坏。自然的，我们会关心模型输出与实际输出的差。所以定义误差为：

\begin{equation}
\epsilon(t,\theta)=y(t)-\phi^T(t)\theta
\end{equation}

为后续计算的方便，取

$$V_N(\theta,Z^N)=\frac{1}{2N}\sum_{t=1}^N[\epsilon]^2$$

为指标函数。上式$V_N$对$\theta$求偏导数，并使用$\phi\phi^T\theta=\phi y$，可以得出对$\theta$的最小二乘估计$\hat{\theta}$。

$$\hat{\theta}=\arg_\theta\min V_N(\theta,Z^N)=[\frac{1}{N}\sum_{t=1}^N\phi(t)\phi^T(t)]^{-1}\frac{1}{N}\sum_{t=1}^N\phi(t)y(t)$$

*注意：这里处理的是SISO情形，即$y(t),u(t)$均是标量。*

定义：

$$R(N)=\frac{1}{N}\sum_{t=1}^N\phi(t)\phi^T(t)\in R^{d\times d}$$

$$f(N)=\frac{1}{N}\sum_{t=1}^N\phi(t)y(t)\in R^d$$


### 1.3 Weighted Least-Squares Estimation(WLS)

如果系统参数时变较快，对过去的输入-输出的拟合意义不大。所以在指标函数中引入权重，衰减过去数据的影响。

修改指标函数为：

$$V_N(\theta,Z^N)=\frac{1}{2N}\sum_{t=1}^N[\alpha_i\epsilon]^2$$(权重固定)

或

$$V_N(\theta,Z^N)=\frac{1}{2N}\sum_{t=1}^N[\beta(N,t)\epsilon]^2$$（权重时变）

## 2 Recursive Least-Squares Algorithm

### 2.1 Prototype of RLS

采用带有权重的最小二乘准则：

$$\hat{\theta_t}=\arg_\theta\min\sum_{k=1}^{t}\beta(t,k)[y(k)-\phi^T(k)\theta]^2$$

可以得到最优估计为：

$$\hat{\theta_t}=\bar{R}^{-1}(t)f(t)$$

其中：

$$\bar{R}(t)=\sum_{k=1}^t\beta(t,k)\phi(k)\phi^T(k)$$

$$f(t)=\sum_{k=1}^t\beta(t,k)\phi(k)y(k)$$

设权重序列满足：

$$\beta(t,k)=\lambda(t)\beta(t-1,k)$$

$$\beta(t,t)=1$$

即：

$$\beta(t,k)=\prod_{j=k+1}^t\lambda(t)$$

*注意：这里处理一种特殊的权重序列。*

在上述权重序列的影响下，回顾$\bar{R}(t),f(t)$的定义式，$\bar{R}(t),f(t)$满足关系：

$$\bar{R}(t)=\lambda(t)\bar{R}(t-1)+\phi(t)\phi^T(t)$$

$$f(t)=\lambda(t)f(t-1)+\phi(t)y(t)$$

将上述关系带入$\hat{\theta}_t$的估计式中：

\begin{align}
\hat{\theta}_t & = \bar{R}(t)^{-1}f(t) \\
& = \bar{R}(t)^{-1}[\lambda(t)f(t-1)+\phi(t)y(t)] \\
& = \bar{R}(t)^{-1}[\lambda(t)\bar{R}(t-1)\hat{\theta}_{t-1}+\phi(t)y(t)] \\
& = \bar{R}(t)^{-1}[\lambda(t)(\bar{R}(t)-\phi(t)\phi^T(t))\hat{\theta}_{t-1}+\phi(t)y(t)] \\
& = \hat{\theta}_{t-1} + \bar{R}^{-1}(t)\phi(t)[y(t)-\phi^T(t)\hat{\theta}_{t-1}]
\end{align}

这样我们就得到了**RLS的两个递推式**：

\begin{eqnarray}
& \hat{\theta}_t & = \hat{\theta}_{t-1} + \bar{R}^{-1}(t)\phi(t)[y(t)-\phi^T(t)\hat{\theta}_{t-1}] \\
& \bar{R}(t) & = \lambda(t)\bar{R}(t-1)+\phi(t)\phi^T(t)
\end{eqnarray}

上式要求计算矩阵的逆，在计算机中计算复杂度较高。所以需要尝试将求逆的操作转化为一个计算较为简便的操作。

引入定理：

$$\exists A,B,C,D:P=A+BCD \rightarrow P^{-1}=[A+BCD]^{-1}=A^{-1}-A^{-1}B[DA^{-1}B+C^{-1}]^{-1}DA^{-1}$$

其中$A,B,C,D,P$均为适当维矩阵。

令$P(t)=\bar{R}^{-1}(t)$，使用关系式凑上述定理的形式，则有$A=\lambda(t)\bar{R}(t-1), B=\phi(t), C=I, D=\phi^T(t)$，
则

\begin{align}
P(t) &= A^{-1}-A^{-1}B[DA^{-1}B+C^{-1}]^{-1}DA^{-1} \\
    &= \frac{1}{\lambda(t)}[P(t-1)-\frac{P(t-1)\phi(t)\phi^T(t)P(t-1)}{\lambda(t)+\phi^T(t)P(t-1)\phi(t)}]
\end{align}

上面的迭代式还可以再进行简化，考虑$P(t)\phi(t)$：

\begin{align}
P(t)\phi(t) &= \frac{1}{\lambda(t)}P(t-1)\phi(t)-\frac{1}{\lambda(t)}\frac{P(t-1)\phi(t)\phi^T(t)P(t-1)\phi(t)}{\lambda(t)+\phi^T(t)P(t-1)\phi(t)} \\
    &=\frac{P(t-1)\phi(t)}{\lambda(t)+\phi^T(t)P(t-1)\phi(t)}
\end{align}

利用上述结果，令$L(t)=P(t)\phi(t)$，我们就获得了带权重的RLS估计算法（也可以称带遗忘因子的RLS估计算法）：

>带权重的RLS估计算法：
\begin{eqnarray}
& L(t) &= \frac{P(t-1)\phi(t)}{\lambda(t)+\phi^T(t)P(t-1)\phi(t)} \\
& \hat{\theta}_t & = \hat{\theta}_{t-1} + L(t)[y(t)-\phi^T(t)\hat{\theta}_{t-1}] \\
& P(t) &= \frac{1}{\lambda(t)}[P(t-1)-L(t)\phi^T(t)P(t-1)] \\
\end{eqnarray}

